In [1]:
!pip install torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from nltk.lm import Vocabulary
import torch
import torch.utils.data as tud
import sys
sys.path.append("../../lib")
#from metrics import levenshtein
import pickle
from pathlib import Path
import re

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
folder = Path("./post_ocr_correction/data/en/data/")

In [3]:
train = pd.read_pickle(folder/"train_aligned_new_pages2.pkl")
train.shape

(234611, 2)

In [4]:
dev = pd.read_pickle(folder/"dev_aligned_new_pages2.pkl")
dev.shape

(389, 2)

In [5]:
with open(folder/"vocabulary_new_pages2.pkl", "rb") as file:
    vocabulary = pickle.load(file)

In [6]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
len(char2i)

37

In [7]:
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
len(i2char)

37

In [8]:
length = 100

output = []
for s in tqdm(train.source):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
train_source = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(train_source.shape)

output = []
for s in tqdm(train.target):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
train_target = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(train_target.shape)

  0%|          | 0/234611 [00:00<?, ?it/s]

torch.Size([234611, 80])


  0%|          | 0/234611 [00:00<?, ?it/s]

torch.Size([234611, 80])


In [9]:
train.source[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in train_source[0].tolist()]))

True

In [10]:
train.target[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in train_target[0].tolist()]))

True

In [11]:
output = []
for s in tqdm(dev.source):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
dev_source = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(dev_source.shape)

output = []
for s in tqdm(dev.target):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
dev_target = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(dev_target.shape)

  0%|          | 0/389 [00:00<?, ?it/s]

torch.Size([389, 80])


  0%|          | 0/389 [00:00<?, ?it/s]

torch.Size([389, 80])


In [12]:
dev.source[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in dev_source[0].tolist()]))

True

In [13]:
dev.target[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in dev_target[0].tolist()]))

True

In [14]:
torch.save(train_source, folder/"train_source_new_pages2.pt")

In [15]:
torch.save(train_target, folder/"train_target_new_pages2.pt")

In [16]:
torch.save(dev_source, folder/"dev_source_new_pages2.pt")

In [17]:
torch.save(dev_target, folder/"dev_target_new_pages2.pt")

In [18]:
with open(folder/"char2i_new_pages2.pkl", "wb") as file:
    pickle.dump(char2i, file)

In [19]:
with open(folder/"i2char_new_pages2.pkl", "wb") as file:
    pickle.dump(i2char, file)